<a href="https://colab.research.google.com/github/Iqra-01/Google_AI_studio_Genai/blob/main/templates/add%20unit%20tests%20to%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkhlcmUncyBhIFB5dGhvbiBmdW5jdGlvbiB0aGF0IGNoZWNrcyB3aGV0aGVyIGEgd29yZCBpcyBhIHBhbGluZHJvbWUuXFxuQWRkIDEwIGRpZmZlcmVudCB1bml0IHRlc3RzIGZvciB0aGlzIHNuaXBwZXQgd2l0aCBjb21tZW50cy4gRm9sbG93IHVwIHdpdGggYSBzaG9ydCBleHBsYW5hdGlvbiBvZiB3aGF0IHdhcyBkb25lLlxcblxcblxcbmBgYFxcbmRlZiBpc19wYWxpbmRyb21lKHdvcmQpOlxcbiAgXFxcIlxcXCJcXFwiXFxuICBDaGVja3Mgd2hldGhlciBhIHdvcmQgaXMgYSBwYWxpbmRyb21lLlxcblxcbiAgQXJnczpcXG4gICAgd29yZDogVGhlIHdvcmQgdG8gY2hlY2suXFxuXFxuICBSZXR1cm5zOlxcbiAgICBUcnVlIGlmIHRoZSB3b3JkIGlzIGEgcGFsaW5kcm9tZSwgRmFsc2Ugb3RoZXJ3aXNlLlxcbiAgXFxcIlxcXCJcXFwiXFxuXFxuICAjIENvbnZlcnQgdGhlIHdvcmQgdG8gbG93ZXJjYXNlIGFuZCByZW1vdmUgbm9uLWFscGhhbnVtZXJpYyBjaGFyYWN0ZXJzLlxcbiAgd29yZCA9ICcnLmpvaW4oY2hhci5sb3dlcigpIGZvciBjaGFyIGluIHdvcmQgaWYgY2hhci5pc2FsbnVtKCkpXFxuXFxuICAjIENoZWNrIGlmIHRoZSB3b3JkIGlzIHRoZSBzYW1lIGZvcndhcmRzIGFuZCBiYWNrd2FyZHMuXFxuICByZXR1cm4gd29yZCA9PSB3b3JkWzo6LTFdXFxuYGBgIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiYGBgcHl0aG9uXFxuZGVmIGlzX3BhbGluZHJvbWUod29yZCk6XFxuICBcXFwiXFxcIlxcXCJcXG4gIENoZWNrcyB3aGV0aGVyIGEgd29yZCBpcyBhIHBhbGluZHJvbWUuXFxuXFxuICBBcmdzOlxcbiAgICB3b3JkOiBUaGUgd29yZCB0byBjaGVjay5cXG5cXG4gIFJldHVybnM6XFxuICAgIFRydWUgaWYgdGhlIHdvcmQgaXMgYSBwYWxpbmRyb21lLCBGYWxzZSBvdGhlcndpc2UuXFxuICBcXFwiXFxcIlxcXCJcXG5cXG4gICMgQ29udmVydCB0aGUgd29yZCB0byBsb3dlcmNhc2UgYW5kIHJlbW92ZSBub24tYWxwaGFudW1lcmljIGNoYXJhY3RlcnMuXFxuICB3b3JkID0gJycuam9pbihjaGFyLmxvd2VyKCkgZm9yIGNoYXIgaW4gd29yZCBpZiBjaGFyLmlzYWxudW0oKSlcXG5cXG4gICMgQ2hlY2sgaWYgdGhlIHdvcmQgaXMgdGhlIHNhbWUgZm9yd2FyZHMgYW5kIGJhY2t3YXJkcy5cXG4gIHJldHVybiB3b3JkID09IHdvcmRbOjotMV1cXG5cXG4jIFVuaXQgVGVzdHNcXG5pbXBvcnQgdW5pdHRlc3RcXG5cXG5jbGFzcyBUZXN0UGFsaW5kcm9tZSh1bml0dGVzdC5UZXN0Q2FzZSk6XFxuXFxuICBkZWYgdGVzdF9lbXB0eV9zdHJpbmcoc2VsZik6XFxuICAgIFxcXCJcXFwiXFxcIlRlc3Qgd2l0aCBhbiBlbXB0eSBzdHJpbmcuXFxcIlxcXCJcXFwiXFxuICAgIHNlbGYuYXNzZXJ0VHJ1ZShpc19wYWxpbmRyb21lKFxcXCJcXFwiKSlcXG5cXG4gIGRlZiB0ZXN0X3NpbmdsZV9jaGFyYWN0ZXIoc2VsZik6XFxuICAgIFxcXCJcXFwiXFxcIlRlc3Qgd2l0aCBhIHNpbmdsZSBjaGFyYWN0ZXIuXFxcIlxcXCJcXFwiXFxuICAgIHNlbGYuYXNzZXJ0VHJ1ZShpc19wYWxpbmRyb21lKFxcXCJhXFxcIikpXFxuXFxuICBkZWYgdGVzdF9wYWxpbmRyb21lX2xvd2VyY2FzZShzZWxmKTpcXG4gICAgXFxcIlxcXCJcXFwiVGVzdCB3aXRoIGEgcGFsaW5kcm9tZSBpbiBsb3dlcmNhc2UuXFxcIlxcXCJcXFwiXFxuICAgIHNlbGYuYXNzZXJ0VHJ1ZShpc19wYWxpbmRyb21lKFxcXCJyYWNlY2FyXFxcIikpXFxuXFxuICBkZWYgdGVzdF9wYWxpbmRyb21lX3VwcGVyY2FzZShzZWxmKTpcXG4gICAgXFxcIlxcXCJcXFwiVGVzdCB3aXRoIGEgcGFsaW5kcm9tZSBpbiB1cHBlcmNhc2UuXFxcIlxcXCJcXFwiXFxuICAgIHNlbGYuYXNzZXJ0VHJ1ZShpc19wYWxpbmRyb21lKFxcXCJSQUNFQ0FSXFxcIikpXFxuXFxuICBkZWYgdGVzdF9wYWxpbmRyb21lX21peGVkX2Nhc2Uoc2VsZik6XFxuICAgIFxcXCJcXFwiXFxcIlRlc3Qgd2l0aCBhIHBhbGluZHJvbWUgaW4gbWl4ZWQgY2FzZS5cXFwiXFxcIlxcXCJcXG4gICAgc2VsZi5hc3NlcnRUcnVlKGlzX3BhbGluZHJvbWUoXFxcIlJhQ2VDYVJcXFwiKSlcXG5cXG4gIGRlZiB0ZXN0X25vbl9wYWxpbmRyb21lX2xvd2VyY2FzZShzZWxmKTpcXG4gICAgXFxcIlxcXCJcXFwiVGVzdCB3aXRoIGEgbm9uLXBhbGluZHJvbWUgaW4gbG93ZXJjYXNlLlxcXCJcXFwiXFxcIlxcbiAgICBzZWxmLmFzc2VydEZhbHNlKGlzX3BhbGluZHJvbWUoXFxcImhlbGxvXFxcIikpXFxuXFxuICBkZWYgdGVzdF9ub25fcGFsaW5kcm9tZV91cHBlcmNhc2Uoc2VsZik6XFxuICAgIFxcXCJcXFwiXFxcIlRlc3Qgd2l0aCBhIG5vbi1wYWxpbmRyb21lIGluIHVwcGVyY2FzZS5cXFwiXFxcIlxcXCJcXG4gICAgc2VsZi5hc3NlcnRGYWxzZShpc19wYWxpbmRyb21lKFxcXCJXT1JMRFxcXCIpKVxcblxcbiAgZGVmIHRlc3Rfbm9uX3BhbGluZHJvbWVfbWl4ZWRfY2FzZShzZWxmKTpcXG4gICAgXFxcIlxcXCJcXFwiVGVzdCB3aXRoIGEgbm9uLXBhbGluZHJvbWUgaW4gbWl4ZWQgY2FzZS5cXFwiXFxcIlxcXCJcXG4gICAgc2VsZi5hc3NlcnRGYWxzZShpc19wYWxpbmRyb21lKFxcXCJIZUxsT1xcXCIpKVxcblxcbiAgZGVmIHRlc3RfcGFsaW5kcm9tZV93aXRoX25vbl9hbHBoYW51bWVyaWMoc2VsZik6XFxuICAgIFxcXCJcXFwiXFxcIlRlc3Qgd2l0aCBhIHBhbGluZHJvbWUgY29udGFpbmluZyBub24tYWxwaGFudW1lcmljIGNoYXJhY3RlcnMuXFxcIlxcXCJcXFwiXFxuICAgIHNlbGYuYXNzZXJ0VHJ1ZShpc19wYWxpbmRyb21lKFxcXCJBIG1hbiwgYSBwbGFuLCBhIGNhbmFsOiBQYW5hbWFcXFwiKSlcXG5cXG4gIGRlZiB0ZXN0X25vbl9wYWxpbmRyb21lX3dpdGhfbm9uX2FscGhhbnVtZXJpYyhzZWxmKTpcXG4gICAgXFxcIlxcXCJcXFwiVGVzdCB3aXRoIGEgbm9uLXBhbGluZHJvbWUgY29udGFpbmluZyBub24tYWxwaGFudW1lcmljIGNoYXJhY3RlcnMuXFxcIlxcXCJcXFwiXFxuICAgIHNlbGYuYXNzZXJ0RmFsc2UoaXNfcGFsaW5kcm9tZShcXFwiVGhpcyBpcyBub3QgYSBwYWxpbmRyb21lLlxcXCIpKVxcblxcbmlmIF9fbmFtZV9fID09ICdfX21haW5fXyc6XFxuICB1bml0dGVzdC5tYWluKClcXG5gYGBcXG5cXG4qKkV4cGxhbmF0aW9uOioqXFxuXFxuMS4gKipJbXBvcnQgYHVuaXR0ZXN0YDoqKiBUaGUgYHVuaXR0ZXN0YCBtb2R1bGUgaXMgaW1wb3J0ZWQgZm9yIHdyaXRpbmcgYW5kIHJ1bm5pbmcgdW5pdCB0ZXN0cy5cXG4yLiAqKkNyZWF0ZSBhIFRlc3QgQ2xhc3M6KiogQSB0ZXN0IGNsYXNzIGBUZXN0UGFsaW5kcm9tZWAgaXMgZGVmaW5lZCwgaW5oZXJpdGluZyBmcm9tIGB1bml0dGVzdC5UZXN0Q2FzZWAuXFxuMy4gKipEZWZpbmUgVGVzdCBNZXRob2RzOioqIEVhY2ggdGVzdCBjYXNlIGlzIGltcGxlbWVudGVkIGFzIGEgbWV0aG9kIHdpdGhpbiB0aGUgY2xhc3MuIEVhY2ggbWV0aG9kIG5hbWUgc3RhcnRzIHdpdGggYHRlc3RfYCB0byBiZSByZWNvZ25pemVkIGFzIGEgdGVzdCBieSBgdW5pdHRlc3RgLlxcbjQuICoqQXNzZXJ0aW9uczoqKiBJbnNpZGUgZWFjaCB0ZXN0IG1ldGhvZCwgYXNzZXJ0aW9ucyBhcmUgdXNlZCB0byBjaGVjayB0aGUgZXhwZWN0ZWQgb3V0Y29tZS4gVGhlIGBzZWxmLmFzc2VydFRydWUoKWAgbWV0aG9kIGNoZWNrcyBpZiB0aGUgcmVzdWx0IG9mIHRoZSBgaXNfcGFsaW5kcm9tZSgpYCBmdW5jdGlvbiBtYXRjaGVzIHRoZSBleHBlY3RlZCBvdXRjb21lIGZvciBwYWxpbmRyb21lIHdvcmRzLiBUaGUgYHNlbGYuYXNzZXJ0RmFsc2UoKWAgbWV0aG9kIGlzIHVzZWQgZm9yIG5vbi1wYWxpbmRyb21lIHdvcmRzLlxcbjUuICoqVGVzdCBDYXNlczoqKiBUZW4gZGlmZmVyZW50IHVuaXQgdGVzdHMgYXJlIHdyaXR0ZW4gdG8gY292ZXIgdmFyaW91cyBzY2VuYXJpb3M6XFxuICAgLSBFbXB0eSBzdHJpbmdcXG4gICAtIFNpbmdsZSBjaGFyYWN0ZXJcXG4gICAtIFBhbGluZHJvbWUgaW4gbG93ZXJjYXNlXFxuICAgLSBQYWxpbmRyb21lIGluIHVwcGVyY2FzZVxcbiAgIC0gUGFsaW5kcm9tZSBpbiBtaXhlZCBjYXNlXFxuICAgLSBOb24tcGFsaW5kcm9tZSBpbiBsb3dlcmNhc2VcXG4gICAtIE5vbi1wYWxpbmRyb21lIGluIHVwcGVyY2FzZVxcbiAgIC0gTm9uLXBhbGluZHJvbWUgaW4gbWl4ZWQgY2FzZVxcbiAgIC0gUGFsaW5kcm9tZSB3aXRoIG5vbi1hbHBoYW51bWVyaWMgY2hhcmFjdGVyc1xcbiAgIC0gTm9uLXBhbGluZHJvbWUgd2l0aCBub24tYWxwaGFudW1lcmljIGNoYXJhY3RlcnNcXG42LiAqKlJ1biBUZXN0czoqKiBUaGUgYHVuaXR0ZXN0Lm1haW4oKWAgZnVuY3Rpb24gcnVucyB0aGUgdGVzdHMgd2hlbiB0aGUgc2NyaXB0IGlzIGV4ZWN1dGVkLiBcXG5cXG5UaGlzIHNldCBvZiB1bml0IHRlc3RzIGVuc3VyZXMgdGhhdCB0aGUgYGlzX3BhbGluZHJvbWUoKWAgZnVuY3Rpb24gY29ycmVjdGx5IGhhbmRsZXMgZGlmZmVyZW50IGlucHV0IGNhc2VzLCBpbmNsdWRpbmcgZW1wdHkgc3RyaW5ncywgc2luZ2xlIGNoYXJhY3RlcnMsIHZhcmlvdXMgY2FzZXMsIGFuZCBub24tYWxwaGFudW1lcmljIGNoYXJhY3RlcnMuIFxcbiJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "Here's a Python function that checks whether a word is a palindrome.\\nAdd 10 different unit tests for this snippet with comments. Follow up with a short explanation of what was done.\\n\\n\\n```\\ndef is_palindrome(word):\\n  \\\"\\\"\\\"\\n  Checks whether a word is a palindrome.\\n\\n  Args:\\n    word: The word to check.\\n\\n  Returns:\\n    True if the word is a palindrome, False otherwise.\\n  \\\"\\\"\\\"\\n\\n  # Convert the word to lowercase and remove non-alphanumeric characters.\\n  word = ''.join(char.lower() for char in word if char.isalnum())\\n\\n  # Check if the word is the same forwards and backwards.\\n  return word == word[::-1]\\n```"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "```python\\ndef is_palindrome(word):\\n  \\\"\\\"\\\"\\n  Checks whether a word is a palindrome.\\n\\n  Args:\\n    word: The 

## Call `generate_content`

In [3]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

This ensures that the function is robust and reliable for any input word.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [4]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Here's a Python function that checks whether a word is a palindrome.\nAdd 10 different unit tests for this snippet with comments. Follow up with a short explanation of what was done.\n\n\n```\ndef is_palindrome(word):\n  \"\"\"\n  Checks whether a word is a palindrome.\n\n  Args:\n    word: The word to check.\n\n  Returns:\n    True if the word is a palindrome, False otherwise.\n  \"\"\"\n\n  # Convert the word to lowercase and remove non-alphanumeric characters.\n  word = ''.join(char.lower() for char in word if char.isalnum())\n\n  # Check if the word is the same forwards and backwards.\n  return word == word[::-1]\n``` 

-------------------------------------------------------------------------------- 

Role: model 

```python\ndef is_palindrome(word):\n  \"\"\"\n  Checks whether a word is a palindrome.\n\n  Args:\n    word: The word to check.\n\n  Returns:\n    True if the word is a palindrome, False otherwise.\n  \"\"\"\n\n  # Convert the word to lowercase and remove n